### TensorFlow Ranking

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from importlib import reload
import seaborn as sns
%load_ext autoreload
%autoreload 2
%matplotlib inline

from dataprep.dataprep import DataPrep
from rank.ranker import Ranker
from pipeline.pipeline import Pipeline

In [2]:
test_data = pd.read_csv(r'C:\Users\afragkoulis\PyCharmProjects\sortranking\data\test.csv')
train_data = pd.read_csv(r'C:\Users\afragkoulis\PyCharmProjects\sortranking\data\train.csv')

In [3]:
raw_train_data = train_data.copy()
raw_test_data = test_data.copy()

In [4]:
train_dataprep = DataPrep(raw_train_data)
test_dataprep = DataPrep(raw_test_data)
df_train = train_dataprep.return_df()
df_test = test_dataprep.return_df()

In [33]:
df_train.to_csv('train_transformed.csv')
df_test.to_csv('test_transformed.csv')

In [15]:
df_test.groupby(by=df_test.index).size().max()

54

In [14]:
df_train.groupby(by=df_train.index).size().max()

54

In [1]:
import tensorflow as tf
import tensorflow_ranking as tfr

tf.enable_eager_execution()
tf.executing_eagerly()

True

In [2]:
_TRAIN_DATA_PATH=r'C:\Users\afragkoulis\PyCharmProjects\sortranking\train_transformed.csv'
_TEST_DATA_PATH=r'C:\Users\afragkoulis\PyCharmProjects\sortranking\test_transformed.csv'
_LOSS="approx_ndcg_loss"
_LIST_SIZE=54
_NUM_FEATURES=39
_BATCH_SIZE=32
_HIDDEN_LAYER_DIMS=["20", "10"]
_LABEL_FEATURE = "prop_booking_bool"

In [3]:
def input_fn(path):
  train_dataset = tf.data.Dataset.from_generator(
      tfr.data.libsvm_generator(path, _NUM_FEATURES, _LIST_SIZE),
      output_types=(
          {str(k): tf.float32 for k in range(1,_NUM_FEATURES+1)},
          tf.float32
      ),
      output_shapes=(
          {str(k): tf.TensorShape([_LIST_SIZE, 1])
            for k in range(1,_NUM_FEATURES+1)},
          tf.TensorShape([_LIST_SIZE])
      )
  )

  train_dataset = train_dataset.shuffle(1000).repeat().batch(_BATCH_SIZE)
  return train_dataset.make_one_shot_iterator().get_next()

In [4]:
def example_feature_columns():
  """Returns the example feature columns."""
  feature_names = [
      "%d" % (i + 1) for i in range(0, _NUM_FEATURES)
  ]
  return {
      name: tf.feature_column.numeric_column(
          name, shape=(1,), default_value=0.0) for name in feature_names
  }

def make_score_fn():
  """Returns a scoring function to build `EstimatorSpec`."""

  def _score_fn(context_features, group_features, mode, params, config):
    """Defines the network to score a documents."""
    del params
    del config
    # Define input layer.
    example_input = [
        tf.layers.flatten(group_features[name])
        for name in sorted(example_feature_columns())
    ]
    input_layer = tf.concat(example_input, 1)

    cur_layer = input_layer
    for i, layer_width in enumerate(int(d) for d in _HIDDEN_LAYER_DIMS):
      cur_layer = tf.layers.dense(
          cur_layer,
          units=layer_width,
          activation="tanh")

    logits = tf.layers.dense(cur_layer, units=1)
    return logits

  return _score_fn

In [5]:
def eval_metric_fns():
  """Returns a dict from name to metric functions.

  This can be customized as follows. Care must be taken when handling padded
  lists.

  def _auc(labels, predictions, features):
    is_label_valid = tf_reshape(tf.greater_equal(labels, 0.), [-1, 1])
    clean_labels = tf.boolean_mask(tf.reshape(labels, [-1, 1], is_label_valid)
    clean_pred = tf.boolean_maks(tf.reshape(predictions, [-1, 1], is_label_valid)
    return tf.metrics.auc(clean_labels, tf.sigmoid(clean_pred), ...)
  metric_fns["auc"] = _auc

  Returns:
    A dict mapping from metric name to a metric function with above signature.
  """
  metric_fns = {}
  metric_fns.update({
      "metric/ndcg@%d" % topn: tfr.metrics.make_ranking_metric_fn(
          tfr.metrics.RankingMetricKey.NDCG, topn=topn)
      for topn in [50]
  })

  return metric_fns

In [6]:
def get_estimator(hparams):
  """Create a ranking estimator.
  Args:
    hparams: (tf.contrib.training.HParams) a hyperparameters object.
  Returns:
    tf.learn `Estimator`.
  """
  def _train_op_fn(loss):
    """Defines train op used in ranking head."""
    return tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.train.get_global_step(),
        learning_rate=hparams.learning_rate,
        optimizer="Adagrad")

  ranking_head = tfr.head.create_ranking_head(
      loss_fn=tfr.losses.make_loss_fn(_LOSS),
      eval_metric_fns=eval_metric_fns(),
      train_op_fn=_train_op_fn)

  return tf.estimator.Estimator(
      model_fn=tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          group_size=1,
          transform_fn=None,
          ranking_head=ranking_head),
      params=hparams)

In [7]:
hparams = tf.contrib.training.HParams(learning_rate=0.05)
ranker = get_estimator(hparams)

C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
W0810 19:05:20.202970 22560 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0810 19:05:20.202970 22560 estimator.py:1811] Using temporary folder as model directory: C:\Users\AFRAGK~1\AppData\Local\Temp\tmp8ewfj48x


In [8]:
tf.reset_default_graph()
ranker.train(input_fn=lambda: input_fn(_TRAIN_DATA_PATH), steps=100)

W0810 19:05:20.220137 22560 deprecation.py:323] From C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\training\training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0810 19:05:20.344852 22560 deprecation.py:323] From C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py:494: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarr

UnknownError: IndexError: list index out of range
Traceback (most recent call last):

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 209, in __call__
    ret = func(*args)

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 514, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_ranking\python\data.py", line 1027, in inner_generator
    qid, doc = _libsvm_parse_line(line)

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_ranking\python\data.py", line 944, in _libsvm_parse_line
    qid = int(tokens[1].split(":")[1])

IndexError: list index out of range


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]

In [32]:
ranker.evaluate(input_fn=lambda: input_fn(_TEST_DATA_PATH), steps=100)

W0810 18:19:10.118621 29188 deprecation.py:323] From C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_ranking\python\metrics.py:154: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
W0810 18:19:10.561619 29188 deprecation.py:323] From C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


UnknownError: IndexError: list index out of range
Traceback (most recent call last):

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 209, in __call__
    ret = func(*args)

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 514, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_ranking\python\data.py", line 1027, in inner_generator
    qid, doc = _libsvm_parse_line(line)

  File "C:\Users\afragkoulis\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow_ranking\python\data.py", line 944, in _libsvm_parse_line
    qid = int(tokens[1].split(":")[1])

IndexError: list index out of range


	 [[{{node PyFunc}}]]
	 [[IteratorGetNext]]

In [ ]:
ranker.model_dir

In [ ]:
model_fn = tfr.model.make_groupwise_ranking_fn(
          group_score_fn=make_score_fn(),
          transform_fn=make_transform_fn(),
          group_size=_GROUP_SIZE,
          ranking_head=ranking_head)

In [ ]:
def train_and_eval_fn():
  """Train and eval function used by `tf.estimator.train_and_evaluate`."""
  run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=1000)
  ranker = tf.estimator.Estimator(
      model_fn=model_fn,
      model_dir=_MODEL_DIR,
      config=run_config)

  train_input_fn = lambda: input_fn(_TRAIN_DATA_PATH)
  eval_input_fn = lambda: input_fn(_TEST_DATA_PATH, num_epochs=1)

  train_spec = tf.estimator.TrainSpec(
      input_fn=train_input_fn, max_steps=_NUM_TRAIN_STEPS)
  eval_spec =  tf.estimator.EvalSpec(
          name="eval",
          input_fn=eval_input_fn,
          throttle_secs=15)
  return (ranker, train_spec, eval_spec)

In [ ]:
! rm -rf "/tmp/ranking_model_dir"  # Clean up the model directory.
ranker, train_spec, eval_spec = train_and_eval_fn()
tf.estimator.train_and_evaluate(ranker, train_spec, eval_spec)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir="/tmp/ranking_model_dir" --port 12345